In [1]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

In [3]:
#py.tools.set_credentials_file(username='sdsouza1996', api_key='FdAFl76mmC57xfMO6ANA')

In [4]:
df = pd.read_csv (r'C:\Users\VCC91462\Desktop\Data_projects\Fifa19_Dataset.csv')

In [5]:
columns_to_drop = ['Unnamed: 0','ID', 'Photo', 'Flag','Club Logo', 'Body Type', 'Real Face', 'Jersey Number',
                   'Height', 'Weight','LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
                   'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
                   'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB','StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
                  'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking',
                  'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina','Special','International Reputation','Skill Moves']

In [6]:
df.drop(columns_to_drop, axis = 1, inplace = True)

In [7]:
df['PG'] = df['Potential'] - df['Overall']

In [8]:
def wage_split(x):
    try:
        return int(x.split("K")[0][1:])
    except:
        return 0
df['Wage'] = df['Wage'].apply(lambda x : wage_split(x))

In [9]:
def value_split(x):
    try:
        if 'M' in x:
            return float(x.split("M")[0][1:])
        elif 'K' in x:
            return float(x.split("K")[0][1:])/1000
    except:
        return 0
df['Value'] = df['Value'].apply(lambda x : value_split(x))

In [10]:
def contract_end(x):
    try:
        if x == '2019':
            return ("Yes")
        elif x == '2020':
            return ("Approaching")
    
    except:
        return ("boo")

In [11]:
df['contract_end'] = df['Contract Valid Until'].apply(lambda x : contract_end(x))

In [12]:
df_21 = df[df.Age <=21]

In [ ]:
df_21['Potential'].quantile([.95])

In [14]:
df_hp = df[(df.Potential>=84) & (df.Age <=21)]

In [15]:
hp_count = df_hp.groupby('Club', as_index = False).count()
#hp_count.reset_index(drop = True, inplace = True)

In [17]:
hp_columns_to_drop = ['Age', 'Nationality', 'Overall', 'Potential', 'Value',
       'Wage', 'Preferred Foot', 'Weak Foot', 'Work Rate', 'Position',
       'Joined', 'Loaned From', 'Contract Valid Until', 'Release Clause', 'PG',
       'contract_end']
hp_count.drop(hp_columns_to_drop, axis = 1, inplace = True)

In [18]:
hp_count.rename(columns = {'Name':'Player_Count'}, inplace = True)

In [19]:
hp_value = df_hp.groupby('Club', as_index = False).sum()
#hp_value.reset_index(drop = True, inplace = True)

In [20]:
hp_asset = pd.merge(hp_value, hp_count, on="Club")

In [23]:
fig = px.scatter(hp_asset, x="Value", y="Player_Count", hover_data=['Club'], size = 'Value', color = 'Value',
        title = 'Clubs with Young Assets',
        labels = {"Value":"Estimated Value (in Millions)","Player_Count":"Player Count"})
fig.show()